# Training Machine Learning with Some Models
Hybrid Recommend Technique with Collaborative Filtering and Content-Based Filtering for Improved Recommender System

## Install and import essential libraries

In [1]:
pip install recommenders pandera Flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.7/750.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
# import essential libraries
import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import lightgbm as lgb
import category_encoders as ce
import recommenders.models.lightgbm.lightgbm_utils as lgb_utils
import lightgbm as lgb
import math
tf.get_logger().setLevel('ERROR') # only show error messages
from sklearn.metrics import mean_squared_error
from datetime import datetime
from tempfile import TemporaryDirectory
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print("LightGBM version: {}".format(lgb.__version__))
print(f"Tensorflow version: {tf.__version__}")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


System version: 3.10.12 (main, Mar 22 2024, 16:50:05) [GCC 11.4.0]
Pandas version: 2.0.3
LightGBM version: 4.4.0
Tensorflow version: 2.15.0


## Read dataset

In [3]:
dataset = pd.read_csv('./dataset.csv', encoding='utf-8', dtype={'userID': str}, sep=",")
dataset.head()

,userID,birthday,gender,itemID,itemName,itemPrice,categoryID,rating,timestamp
0,0577980460,1988-12-17,Female,22,Lẩu nấm thập cẩm,157000,6,5.0,1365811200
1,0517030417,NaN,Female,29,Rau thêm,10000,8,NaN,1341100800
2,0265129175,2014-01-24,Female,2,Chả giò,80000,1,NaN,1367193600
3,0631007419,2000-07-23,Male,2,Chả giò,80000,1,3.0,1374451200
4,0484119129,1977-05-22,Female,2,Chả giò,80000,1,1.0,1334707200


## Data Preprocessing

In [4]:
def calculate_age(birthday):
    today = datetime.today()
    birth_date = pd.to_datetime(birthday)
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return age

# Add age column into DataFrame
if 'birthday' in dataset.columns:
  dataset['age'] = dataset['birthday'].apply(calculate_age)

# Remove birthday column
dataset = dataset.drop(columns=['birthday'], errors='ignore')

dataset.head()

,userID,gender,itemID,itemName,itemPrice,categoryID,rating,timestamp,age
0,0577980460,Female,22,Lẩu nấm thập cẩm,157000,6,5.0,1365811200,35.0
1,0517030417,Female,29,Rau thêm,10000,8,NaN,1341100800,NaN
2,0265129175,Female,2,Chả giò,80000,1,NaN,1367193600,10.0
3,0631007419,Male,2,Chả giò,80000,1,3.0,1374451200,24.0
4,0484119129,Female,2,Chả giò,80000,1,1.0,1334707200,47.0


In [5]:
# Encode Categorical Data
dataset['gender'] = dataset['gender'].map({'Female': 0, 'Male': 1})

In [6]:
# Data missing handler
dataset['age'].fillna(round(dataset['age'].mean()), inplace=True)
dataset['age'] = dataset['age'].astype(int) # Convert integer type

# Not every user will rate the dish after order, so it is necessary to handle missing rating data

num_missing_ratings = dataset['rating'].isnull().sum()
if num_missing_ratings > 0:
  from sklearn.impute import KNNImputer
  # Replace the missing value with the average value of that dishes
  feature_user_item = dataset[['userID', 'itemID', 'age', 'gender', 'categoryID', 'itemPrice', 'rating']]
  # For each missing value, KNNImputer finds the k nearest neighbors and calculates a replacement value based on the values ​​of those neighbors. If all user and item features are included, KNNImputer will rely on these characteristics to calculate the distance and fill in missing values.
  imputer = KNNImputer(n_neighbors=5) # Create a KNNImputer object with k = 5
  data_imputed = imputer.fit_transform(feature_user_item) # Apply KNNImputer
  dataset['rating'] = pd.DataFrame(data_imputed)[[6]] # 6 is column's name (as well as index column)
  dataset['rating'] = dataset['rating'].astype(int) # Convert integer type

In [7]:
# Feature selection
featuresUsed = ['userID', 'itemID', 'gender', 'age', 'itemPrice', 'rating', 'categoryID', 'timestamp']
dataset = dataset[featuresUsed]
dataset.head()
#Feature scaling: no need

,userID,itemID,gender,age,itemPrice,rating,categoryID,timestamp
0,0577980460,22,0,35,157000,5,6,1365811200
1,0517030417,29,0,31,10000,3,8,1341100800
2,0265129175,2,0,10,80000,3,1,1367193600
3,0631007419,2,1,24,80000,3,1,1374451200
4,0484119129,2,0,47,80000,1,1,1334707200


In [8]:
# Split dataset
train, test = python_stratified_split(dataset, ratio=0.8)
# print(train)

## Training LightGCN Model

In [11]:
TOP_K = 10 # top k items to recommend

def Setup_LightGCN (train, test):
  SEED = DEFAULT_SEED  # Set None for non-deterministic results
  data = ImplicitCF(train=train, test=test, seed=SEED)
  yaml_file = "./lightgcn.yaml"
  # Model parameters
  EPOCHS = 50
  BATCH_SIZE = 2048
  hparams = prepare_hparams(yaml_file,
                          n_layers=4,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.01,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )
  modelLightGCN = LightGCN(hparams, data, seed=SEED)
  return modelLightGCN

def Training_LightGCN(modelLightGCN):
  with Timer() as train_time:
    modelLightGCN.fit()
  print("Took {} seconds for training.".format(train_time.interval))
  topk_scores = modelLightGCN.recommend_k_items(test, top_k=TOP_K, remove_seen=False) # Remove seen is param to decide hidden item which user have seen
  # print(topk_scores)
  return topk_scores, modelLightGCN

def predict_LightGCN (user_id_predict): # for combine 2 models
  try:
    predict_data = pd.DataFrame({
      'userID': [user_id_predict]
    })
    # predict score for items
    result = modelLightGCN.recommend_k_items(predict_data, top_k=TOP_K, remove_seen=False)
    return result.drop(columns=['userID']) # Drop column to have the same size columns with LightGBM
  except Exception as e:
    print(str(e))
    # return pd.DataFrame(columns = ['itemID', 'prediction'])
    top_10_items = dataset.groupby('itemID')['rating'].mean().nlargest(10).reset_index()
    top_10_items.columns = ['itemID', 'prediction']
    return top_10_items

# Call train, evaluate functions
modelLightGCN = Setup_LightGCN(train, test)
topk_scores, modelLightGCN = Training_LightGCN(modelLightGCN)
#print(predict_LightGCN ('0105289402')) #0435321829 #0111111

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)0.1s: train loss = 0.68623 = (mf)0.68610 + (embed)0.00013
Epoch 2 (train)0.0s: train loss = 0.66939 = (mf)0.66925 + (embed)0.00015
Epoch 3 (train)0.0s: train loss = 0.63968 = (mf)0.63952 + (embed)0.00016
Epoch 4 (train)0.0s: train loss = 0.59700 = (mf)0.59683 + (embed)0.00017
Epoch 5 (train)0.0s + (eval)0.1s: train loss = 0.54027 = (mf)0.54008 + (embed)0.00019, recall = 0.38811, ndcg = 0.31934, precision = 0.03881, map = 0.29755
Epoch 6 (train)0.0s: train loss = 0.47650 = (mf)0.47630 + (embed)0.00020
Epoch 7 (train)0.0s: train loss = 0.41212 = (mf)0.41190 + (embed)0.00022
Epoch 8 (train)0.0s: train loss = 0.34707 = (mf)0.34683 + (embed)0.00023
Epoch 9 (train)0.0s: train loss = 0.28373 = (mf)0.28348 + (embed)0.00025
Epoch 10 (train)0.0s + (eval)0.1s: train loss = 0.23536 = (mf)0.23509 + (embed)0.00027, recall = 0.39860, ndcg = 0.32324, precision = 0.03986, map = 0.29971
Epoch

In [12]:
def Evaluate_LightGCN(topk_scores):
  eval_map = map(test, topk_scores, k=TOP_K)
  eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
  eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
  eval_recall = recall_at_k(test, topk_scores, k=TOP_K)
  print("MAP:\t%f" % eval_map,
        "NDCG:\t%f" % eval_ndcg,
        "Precision@K:\t%f" % eval_precision,
        "Recall@K:\t%f" % eval_recall, sep='\n')

Evaluate_LightGCN(topk_scores)

MAP:	0.647666
NDCG:	0.707833
Precision@K:	0.089161
Recall@K:	0.891608


## Training LightGBM Model

In [13]:
if 'timestamp' in train.columns:
  train.drop(columns=['timestamp'], inplace=True) # Not essential in this model, isn't feature of user or item
if 'timestamp' in test.columns:
  test.drop(columns=['timestamp'], inplace=True) # Not essential in this model, isn't feature of user or item
# dataset.head()
# print(train)
# print(test)

In [14]:
# Create feature and label for model
def Create_Feature_Label (train, test):
  X_train = train.drop(['rating'], axis=1)
  y_train = train['rating']
  X_test = test.drop(['rating'], axis=1)
  y_test = test['rating']

  # The model only accepts numbers, not strings or objects, so the data needs to be converted
  # Convert gender, userID columns in train data into number or float
  X_train['userID'] = X_train['userID'].astype(int)

  # Convert gender, userID columns in test data into number or float
  X_test['userID'] = X_test['userID'].astype(int)

  # Create dataset for LightGBM
  train_dataset = lgb.Dataset(X_train, label=y_train)
  test_dataset = lgb.Dataset(X_test, label=y_test, reference=train_dataset)
  return train_dataset, test_dataset, X_test, y_test

train_dataset, test_dataset, X_test, y_test = Create_Feature_Label(train, test)

In [15]:
# print(train_dataset)
# print(test_dataset)

In [16]:
# Training LightGBM Model
params = {
    'objective': 'regression',
    'metric': 'rmse'
}

modelLightGBM = lgb.train(params, train_dataset, valid_sets=[test_dataset], callbacks=[lgb.early_stopping(10)])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000292 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 366
[LightGBM] [Info] Number of data points in the train set: 1714, number of used features: 6
[LightGBM] [Info] Start training from score 4.151692
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	valid_0's rmse: 1.35664


In [17]:
# Evaluation
def Evaluate_LightGBM(X_test, y_test):
  # predict on test data
  y_pred = modelLightGBM.predict(X_test)

  # Evaluatate model
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  print(f'RMSE: {rmse}')

Evaluate_LightGBM(X_test, y_test)

RMSE: 1.356643323691832


In [18]:
# For combie two models
def predict_LightGBM (user_id_predict, user_gender, user_age_predict):
  user_id_predict = int(user_id_predict)
  user_gender_predict = 0 if user_gender == 'Female' else 1

  df_item_predict = pd.read_json('./data_item.json').drop_duplicates(subset='_id')
  df_item_predict = df_item_predict[['_id', 'price', 'category_id']]
  df_item_predict.columns = ['itemID', 'itemPrice', 'categoryID']
  df_item_predict_length = df_item_predict.shape[0]

  df_user_predict = pd.DataFrame({
      'userID': [user_id_predict] * df_item_predict_length,  # Create n rows with the same user
      'gender': [user_gender_predict] * df_item_predict_length,  # user info
      'age': [user_age_predict] * df_item_predict_length,  # user info
  })

  predict_data = pd.concat([df_user_predict, df_item_predict], axis=1)

  # predict score for items
  predict_data_result = modelLightGBM.predict(predict_data)

  # add prediction columns into df
  predict_data['prediction_lightgbm'] = predict_data_result

  recommended_items = predict_data.sort_values(by='prediction_lightgbm', ascending=False)
  top_N_recommendations = recommended_items.head(TOP_K)
  return top_N_recommendations[['itemID', 'prediction_lightgbm']]

## Combine two models and API integration

In [21]:
def recommend_items(user_id_predict, user_gender, user_age_predict):
  recommend_items_LightGBM = predict_LightGBM(user_id_predict, user_gender, user_age_predict) # print(recommend_items_LightGBM)
  recommend_items_LightGCN = predict_LightGCN(user_id_predict) # print(recommend_items_LightGCN)
  df_combined = pd.merge(recommend_items_LightGCN, recommend_items_LightGBM, on=['itemID'], how='outer')
  df_combined['prediction'].fillna(0, inplace=True)
  df_combined['prediction_lightgbm'].fillna(0, inplace=True)
  df_combined['predicted_score_avg'] = (df_combined['prediction_lightgbm'] + df_combined['prediction']) / 2
  print(df_combined)
  df_combined_sorted = df_combined.sort_values(by='predicted_score_avg', ascending=False) # Sort by predict_score
  # print(df_combined_sorted)
  top_n_items = df_combined_sorted.head(TOP_K) # Choose top-N item to recommend
  return top_n_items['itemID'].values.tolist()

# Testing
# print(recommend_items('0105289402', 'Female', 25)) # 0435321829

    itemID  prediction  prediction_lightgbm  predicted_score_avg
0       12   11.942873             4.365708             8.154291
1       37    7.115560             0.000000             3.557780
2        6    5.210639             0.000000             2.605320
3        9    3.163167             0.000000             1.581583
4        4    1.632548             4.365708             2.999128
5        7    1.243303             0.000000             0.621652
6       17    0.865431             0.000000             0.432715
7       34    0.851159             0.000000             0.425580
8       25    0.840867             0.000000             0.420434
9       11    0.754289             0.000000             0.377144
10      26    0.000000             4.136665             2.068333
11      27    0.000000             4.117075             2.058537
12      28    0.000000             4.117075             2.058537
13      20    0.000000             4.079268             2.039634
14      19    0.000000   

In [ ]:
# API integration
from flask import Flask, jsonify, request
import urllib.parse
import json
import random

app = Flask(__name__)

def generate_random_phone_number():
    return f'0{random.randint(100000000, 999999999)}'

@app.route('/recommend/menu', methods=['GET'])
def handler_recommend_api():
  try:
    item = ''
    header_info = request.headers.get('user-infor-header')
    if header_info is None:
      items = recommend_items(generate_random_phone_number(), random.choice(['Male', 'Female']), random.randint(5, 50))
    else:
      decoded_info = urllib.parse.unquote(header_info)
      user = json.loads(decoded_info)
      if 'birthday' in user:
        age = calculate_age(user['birthday'])
        items = recommend_items(user['_id'], user['gender'], age)
      else:
        items = recommend_items(user['_id'], user['gender'], random.randint(5, 50))
    return jsonify({
        'status': 'success',
        'message': 'Lấy danh sách đề xuất món ăn thành công',
        'data': json.dumps(items)
    }), 200
  except Exception as e:
    return jsonify({
        'status': 'error',
        'message': str(e)
    }), 500

@app.errorhandler(404)
def page_not_found(e):
  return jsonify({
    'status': 'error',
    'message': 'Không tìm thấy đường dẫn'
  }), 404

# Run server
if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, port=5090, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5090
 * Running on http://172.28.0.12:5090
INFO:werkzeug:Press CTRL+C to quit
